## TypedDict

In [6]:
from langchain_openai import ChatOpenAI
from typing import List, Optional, Annotated, TypedDict
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.6)

class Movie(TypedDict):

  title: Annotated[str, "Name of the movie"]
  release_year: Annotated[int, "Year the movie was released"]
  genres: Annotated[List[str], "List of genres the movie belongs to"]
  rating: Annotated[float, "Average rating of the movie on a scale of 1 to 10"]
  box_office: Annotated[Optional[float], "Worldwide box office in millions USD, if known"]

structured_llm = llm.with_structured_output(Movie)

result = structured_llm.invoke("Give me details of a movie named Inception")

# print(result)
# print(type(result))

movie = Movie({"title":12})
print(movie["title"])


12


## Pydantic

In [11]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List, Optional
import dotenv


dotenv.load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.6)

class Movie(BaseModel):
  title: str = Field(..., description="Name of the movie")
  release_year: int = Field(..., description= "Year the movie was released")
  genres: List[str] = Field(..., description= "List of genres the movie belongs to")
  rating: float = Field(..., description= "Average rating of the movie on a scale of 1 to 10")
  box_office: Optional[float] = Field(..., description= "Worldwide box office in millions USD, if known")


structured_llm = llm.with_structured_output(Movie)


result = structured_llm.invoke("Give me details about the movie Inception.")

# print(result)
# print(type(result))
# print(result.model_dump())
# print(result.model_dump_json())

movie = Movie(genres=["Sci-Fi", "Action", "Adventure"], 
              title="Inception", 
              release_year=2010, 
              rating=8.8,
              box_office=None)
print(movie)

title='Inception' release_year=2010 genres=['Sci-Fi', 'Action', 'Adventure'] rating=8.8 box_office=None


## JSON_Schema

In [12]:
from langchain_openai import ChatOpenAI
import dotenv

dotenv.load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ✅ Correct JSON Schema for LangChain / OpenAI function calling
movie_json_schema = {
    "name": "movie_schema",
    "description": "Schema for extracting movie information",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {
                "type": "string",
                "description": "Name of the movie"
            },
            "release_year": {
                "type": "integer",
                "description": "Year the movie was released"
            },
            "genres": {
                "type": "array",
                "items": {"type": "string"},
                "description": "List of genres the movie belongs to"
            },
            "rating": {
                "type": "number",
                "description": "Average rating of the movie on a scale of 1 to 10"
            },
            "box_office": {
                "type": ["number", "null"],
                "description": "Worldwide box office in millions USD, if known"
            }
        },
        "required": ["title", "release_year", "genres", "rating", "box_office"]
    }
}

structured_llm = llm.with_structured_output(movie_json_schema)

result = structured_llm.invoke("Give me details about the movie Inception.")
print(result)

{'title': 'Inception', 'release_year': 2010, 'genres': ['Action', 'Adventure', 'Sci-Fi', 'Thriller'], 'rating': 8.8, 'box_office': 836.8}


In [13]:
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI


class ProductReview(BaseModel):
    product_name: str = Field(..., description="Name of the product being reviewed")
    reviewer_name: str = Field(..., description="Name of the reviewer")
    rating: float = Field(..., description="Product rating given by the reviewer on a scale of 1 to 5")
    pros: List[str] = Field(..., description="List of positive aspects of the product")
    cons: List[str] = Field(..., description="List of negative aspects of the product")
    review_text: str = Field(..., description="Detailed written review of the product")
    would_recommend: bool = Field(..., description="Whether the reviewer recommends the product")
    purchase_date: Optional[str] = Field(None, description="Date when the product was purchased (YYYY-MM-DD)")


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.6)
structured_llm = llm.with_structured_output(ProductReview)

result = structured_llm.invoke("""I recently purchased the Bose QuietComfort 45 Noise Cancelling Headphones about three months ago, and I have been using them almost daily since then. As someone who works from home and also travels frequently, I needed headphones that could block out distractions, deliver excellent audio quality, and remain comfortable for long listening sessions. The QC45s absolutely deliver in most of these aspects.

First, the noise cancellation is outstanding. On a recent flight from New York to London, the cabin noise was reduced to a soft whisper, allowing me to enjoy my music and podcasts without cranking up the volume. Even at home, they help me focus by muting the hum of my air conditioner and the occasional street noise.

The sound quality is top-notch. The bass is deep but not overpowering, the mids are clear, and the highs are crisp. Whether I’m listening to jazz, classical music, or electronic tracks, everything sounds balanced and immersive. Bluetooth pairing is seamless, and the connection has been rock solid even when I move around my apartment.

Comfort-wise, I can wear them for 4–5 hours straight without any ear fatigue. The ear cups are soft and well-cushioned, and the clamping force is just right. The build quality feels premium, with smooth adjustments and a sturdy frame.

Battery life is exceptional — I’ve easily gotten over 22 hours of use on a single charge, and the quick-charge feature is a lifesaver when I forget to plug them in overnight.

However, they are not perfect. The touch controls can be a bit finicky, especially in cold weather when I’m wearing gloves. Additionally, the case feels slightly bulky if you’re trying to travel light.

Overall, these headphones have exceeded my expectations. For anyone looking for premium noise-cancelling headphones for travel, work, or daily use, I would highly recommend them without hesitation.
""")
print(result)



product_name='Bose QuietComfort 45 Noise Cancelling Headphones' reviewer_name='John Doe' rating=4.5 pros=['Outstanding noise cancellation', 'Excellent sound quality', 'Comfortable for long listening sessions', 'Exceptional battery life', 'Seamless Bluetooth pairing'] cons=['Touch controls can be finicky', 'Case is slightly bulky for travel'] review_text='I recently purchased the Bose QuietComfort 45 Noise Cancelling Headphones about three months ago, and I have been using them almost daily since then. As someone who works from home and also travels frequently, I needed headphones that could block out distractions, deliver excellent audio quality, and remain comfortable for long listening sessions. The QC45s absolutely deliver in most of these aspects.\n\nFirst, the noise cancellation is outstanding. On a recent flight from New York to London, the cabin noise was reduced to a soft whisper, allowing me to enjoy my music and podcasts without cranking up the volume. Even at home, they help